In [1]:
import pandas as pd
import numpy as np
import datascience as ds

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# References: Lab02

In [ ]:
# TODO: complete this function
def print_false_discovery_fraction(results_dictionary):
    total_predicted_discoveries = results_dictionary["FP_count"] + results_dictionary["TP_count"]
    false_predicted_discoveries = results_dictionary["FP_count"]

    # TODO: fill in - compute the false discovery fraction from the `results` dictionary
    false_discovery_frac = false_predicted_discoveries / total_predicted_discoveries

    print("total discoveries: {0}".format(total_predicted_discoveries))
    print("fraction of discoveries which were actually false: {0:.3f}".format(false_discovery_frac))
    return total_predicted_discoveries, false_discovery_frac



In [ ]:
hyp1 = [0.000000000100511363239]
hyp2 = [0.999965244371]
hyp3 = [0.999999997186]
#hyp3 = [0.99998613, 0.99999825,  0.99999766, 0.9999974, 0.99999886, 0.99999397]
#hyp4 = [0.999997, 0.999992, 1.000000, 0.999996, 0.999994, 0.999996, 0.999998]
hyp4 = [0.999999995676]
hyp5 = [0.999633172991]
hyp6 = [0.992741487968]

#true_values = [1, 1, 0, 0, 0, 0, 0,0,0,0,0,0]
true_values = [1, 0, 0, 0, 0, 0]

p_values = hyp1 + hyp2 + hyp3 + hyp4 + hyp5 + hyp6
alpha_total = 0.05
alpha = 0.05
n = 6
print(p_values)

[1.00511363239e-10, 0.999965244371, 0.999999997186, 0.999999995676, 0.999633172991, 0.992741487968]


In [ ]:
# TODO: complete this function
def report_results(predicted_discoveries, truth):
    """
    Produces a dictionary with counts for the true positives, true negatives,
    false negatives, and false positives from the input `predicted_discoveries`
    and `truth` arrays.

    Inputs:
      predicted discoveries: array of 0/1 values where 1 indicates a "discovery".
      truth: array of 0/1 values where 1 indicates a draw from the alternative.

    Outputs: a dictionary of TN, TP, FN, and FP counts.
    """

    # Convert lists to arrays
    predicted_discoveries = np.array(predicted_discoveries)
    truth = np.array(truth)

    # Populate the following dictionary with counts (NOT rates)
    TP_count = np.sum(predicted_discoveries * truth)
    TN_count = np.sum((1 - predicted_discoveries) * (1 - truth))
    FP_count = np.sum(predicted_discoveries * (1 - truth))
    FN_count = np.sum((1 - predicted_discoveries) * truth)

    results_dictionary = {
        "TN_count": TN_count,
        "TP_count": TP_count,
        "FN_count": FN_count,
        "FP_count": FP_count
    }

    # This function is defined for you below
    print_confusion_matrix(results_dictionary)
    return results_dictionary

# Bonferroni correction procedure

In [ ]:
# TODO: calculate the decisions based on the bonferroni correction procedure.
def bonferroni(p_values, alpha_total):
    """
    Returns decisions on p-values using the Bonferroni correction.

    Inputs:
        p_values: array of p-values
        alpha_total: desired family-wise error rate (FWER = P(at least one false discovery))

    Returns:
        decisions: binary array of same length as p-values, where `decisions[i]` is 1
        if `p_values[i]` is deemed significant, and 0 otherwise
    """
    alpha = alpha_total / len(p_values)

    decisions = []

    for p_value in p_values:
        if p_value <= alpha:
            decisions.append(1)  # Significant
        else:
            decisions.append(0)  # Not significant

    return decisions

In [ ]:
def print_confusion_matrix(res_dict):
    # This is a helper function to print the confusion matrix. You don't need to modify this code.
    results_df = pd.DataFrame(data = {"Decision = 0": [res_dict['TN_count'], res_dict['FN_count']],
                                      "Decision = 1":  [res_dict['FP_count'], res_dict['TP_count']]},
                             index=["Truth = 0", "Truth = 1"])
    print(results_df)



In [ ]:
# Once you've filled in `bonferroni`, run this cell to print the results.
bonferroni_decisions = bonferroni(p_values, alpha)

bonferroni_results = report_results(bonferroni_decisions,true_values)
print()
print_false_discovery_fraction(bonferroni_results)
print()


           Decision = 0  Decision = 1
Truth = 0             5             0
Truth = 1             0             1

total discoveries: 1
fraction of discoveries which were actually false: 0.000



# Benjamini-Hochberg procedure

In [ ]:
# TODO: calculate decisions based on Benjamini-Hochberg procedure
def benjamini_hochberg(p_values, alpha):
    """
    Returns decisions on p-values using Benjamini-Hochberg.

    Inputs:
        p_values: array of p-values
        alpha: desired FDR (FDR = E[# false positives / # positives])

    Returns:
        decisions: binary array of same length as p-values, where `decisions[i]` is 1
        if `p_values[i]` is deemed significant, and 0 otherwise
    """
    num_p = len(p_values)
    sorted_pval = np.sort(p_values)

    for k in range(num_p):
        if sorted_pval[k] <= (k+1) * (alpha/num_p):
            max_k = k

    threshold = sorted_pval[max_k]

    decisions = p_values <= threshold
    return decisions

In [ ]:
# Once you've filled in `benjamini_hochberg`, run this cell to print the results.
bh_decisions = benjamini_hochberg(p_values, alpha)

bh_results = report_results(bh_decisions,true_values)
print()

print_false_discovery_fraction(bh_results)
print(bh_decisions)


           Decision = 0  Decision = 1
Truth = 0             5             0
Truth = 1             0             1

total discoveries: 1
fraction of discoveries which were actually false: 0.000
[ True False False False False False]
